In [ ]:
# 주피터 노트북 환경설정
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

from IPython.display import Image

from IPython.core.display import display, HTML
# display(HTML("<style>.container { font-weight: bold !important; font-family:'Malgun Gothic' !important;}</style>"))
display(HTML("<style>.container { font-weight: bold !important;}</style>"))
display(HTML("<style>.container { width: 98% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import os

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 11.0
# plt.rcParams['font.family'] = 'batang'
# plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [10, 6]

In [ ]:
from sklearn.ensemble import VotingClassifier, VotingRegressor

from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import f1_score, recall_score, precision_score, classification_report, r2_score, confusion_matrix

from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.pipeline import Pipeline

# 배깅(Bagging)
- Bootstrap Aggregation. 앙상블 학습 방식 중 하나 
- 배깅은 샘플을 여러 번 뽑아(Bootstrap) 각 모델을 학습시켜 결과물을 집계(Aggregration)하는 방법
- 데이터로부터 부트스트랩(복원 랜덤 샘플링)을 하고 부트스트랩한 데이터로 모델을 학습. 학습된 모델의 결과를 집계하여 최종 결과 값을 구한다.
- Categorical Data는 투표 방식(Votinig)으로 결과를 집계하며, Continuous Data는 평균으로 집계.
- Categorical Data일 때, 투표 방식으로 한다는 것은 전체 모델에서 예측한 값 중 가장 많은 값을 최종 예측값으로 선정. 
- 6개의 결정 트리 모델일 경우 4개는 A로 예측했고, 2개는 B로 예측했다면 투표에 의해 4개의 모델이 선택한 A를 최종 결과로 예측한다
- 평균으로 집계한다는 것은 말 그대로 각각의 결정 트리 모델이 예측한 값에 평균을 취해 최종 Bagging Model의 예측값을 결정한다
- 배깅 기법을 활용한 모델이 바로 랜덤 포레스트

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fb4wG8O%2FbtqyfYW98AS%2FYZBtUJy3jZLyuik1R0aGNk%2Fimg.png' width='600'>

# 랜덤 포레스트(Random Forest)

- 여러 개의 분류기를 만들어서 보팅으로 최종 결정하는 알고리즘. 
- 여러 개의 결정 트리 분류기가 전체 데이터에서 배깅 방식으로 각자의 데이터를 샘플링해 개별적으로 학습을 수행한 뒤 최종적으로 모든 분류기가 보팅을 통해 예측 결정을 내린다.
- 개별적인 분류기의 기반 알고리즘은 결정 트리이지만 개별 트리가 학습하는 데이터 세트는 전체 데이터에서 일부가 중첩되게 샘플링된 데이터 세트인데 여러 개의 데이터 세트를 중첩되게 분리하는 것을 부트 스트래핑(bootstrapping) 분할 방식이라고 한다. 

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fod2gM%2Fbtq6kMv83oZ%2FsNVWEiA7FWsB2BBt6kn0r1%2Fimg.png'
      width='500'>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

### 타이타닉 데이타셋

In [ ]:
titanic_df = pd.read_csv('data/titanic_book.csv')

titanic_df['Age'].fillna(titanic_df['Age'].mean(),inplace=True)
titanic_df['Cabin'].fillna('N', inplace=True)
titanic_df['Embarked'].fillna('N', inplace=True)

encoder = LabelEncoder()
encoder.fit(titanic_df['Sex'])
temp = encoder.transform(titanic_df['Sex'])
titanic_df['Sex'] = temp.reshape(-1, 1)

encoder.fit(titanic_df['Embarked'])
temp = encoder.transform(titanic_df['Embarked'])
titanic_df['Embarked'] = temp.reshape(-1, 1)

titanic_df['Cabin'] = titanic_df['Cabin'].str[:1]
encoder.fit(titanic_df['Cabin'])
temp = encoder.transform(titanic_df['Cabin'])
titanic_df['Cabin'] = temp.reshape(-1, 1)


titanic_df.drop(['PassengerId','Name','Ticket'], axis=1, inplace=True)

In [ ]:
titanic_df.head(2)

In [ ]:
titanic_df.isnull().sum()

In [ ]:
X_titanic_df = titanic_df.loc[:, 'Pclass':]
y_titanic_df = titanic_df['Survived']

### 학습 데이타와 테스트 데이타로 분리 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, 
                                                    test_size=0.2, random_state=11,
                                                    stratify=y_titanic_df)

In [ ]:
X_train.shape, X_test.shape

### DecesionTree 모델 적용과 테스트

In [ ]:
model_dt = DecisionTreeClassifier()
model_dt.fit(X_train, y_train)
model_dt.score(X_train, y_train), model_dt.score(X_test, y_test)

In [ ]:
model_dt = DecisionTreeClassifier()
parameters = {'max_depth':[3, 4, 5, 6, 7, 8, 9, 10]}
model_dt_grid = GridSearchCV(model_dt, param_grid=parameters, cv=5,  verbose=1)
model_dt_grid.fit(X_train, y_train)
model_dt_grid.best_params_

In [ ]:
model_dt = DecisionTreeClassifier(max_depth = 3)
model_dt.fit(X_train, y_train)
model_dt.score(X_train, y_train), model_dt.score(X_test, y_test) 

In [ ]:
ftr_importances = pd.Series(model_dt.feature_importances_ , index=X_train.columns  )
plt.figure(figsize=(8,6))
plt.title('Feature importances')
sns.barplot(x=ftr_importances , y = ftr_importances.index)
plt.show()

### Random Forest 적용

In [ ]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)
model_rf.score(X_train, y_train), model_rf.score(X_test, y_test)

In [ ]:
model_rf = DecisionTreeClassifier()
parameters = {'max_depth':[3, 4, 5, 6, 7, 8, 9, 10]}
model_rf_grid = GridSearchCV(model_rf, param_grid=parameters, cv=5,  verbose=1)
model_rf_grid.fit(X_train, y_train)
model_rf_grid.best_params_

In [ ]:
model_rf = RandomForestClassifier(max_depth = 3)
model_rf.fit(X_train, y_train)
model_rf.score(X_test, y_test)

In [ ]:
model_rf.classes_ , model_rf.feature_importances_, X_train.columns

In [ ]:
importances_values = model_rf.feature_importances_
print(np.argsort(importances_values)[::-1])
X_train.columns[np.argsort(importances_values)[::-1][:]]

In [ ]:
ftr_importances = pd.Series(model_rf.feature_importances_ , index=X_train.columns  )
plt.figure(figsize=(8,6))
plt.title('Feature importances')
sns.barplot(x=ftr_importances , y = ftr_importances.index)
plt.show()

# 부스팅(Boosting)
- 부스팅은 가중치를 활용하여 약 분류기를 강 분류기로 만드는 방법. 
- 배깅은 Deicison Tree1과 Decision Tree2가 서로 독립적으로 결과를 예측하며 여러 개의 독립적인 결정 트리가 각각 값을 예측한 뒤, 그 결과 값을 집계해 최종 결과 값을 예측하는 방식.
- 부스팅은 모델 간 팀워크가 이루어져 처음 모델이 예측을 하면 그 예측 결과에 따라 데이터에 가중치가 부여되고, 부여된 가중치가 다음 모델에 영향을 준다. 잘못 분류된 데이터에 집중하여 새로운 분류 규칙을 만드는 단계를 반복한다. 
- ADA Boosting, Gradient Boosting

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FkCejr%2FbtqyghvqEZB%2F9o3rKTEsuSIDHEfelYFJlk%2Fimg.png' width='600'>



# GBM

- Gradient Boosting Machine

- max_depth	: 트리의 최대 깊이. default = 3
- min_samples_split	: 노드를 분할하기 위한 최소한의 샘플 데이터수. → 과적합을 제어하는데 사용
    <br> Default = 2 → 작게 설정할 수록 분할 노드가 많아져 과적합 가능성 증가
- min_samples_leaf : 리프노드가 되기 위해 필요한 최소한의 샘플 데이터수.  
    <br> min_samples_split과 함께 과적합 제어 용도.  default = 1
- max_features : 최적의 분할을 위해 고려할 최대 feature 개수.  Default = 'none' → 모든 피처 사용
- max_leaf_nodes : 리프노드의 최대 개수.  default = None → 제한없음
- n_estimators	: 생성할 트리의 갯수. Default = 100
- learning_rate	: 학습을 진행할 때마다 적용하는 학습률(0~1). 
    <br> Weak learner가 순차적으로 오류 값을 보정해나갈 때 적용하는 계수.  Default = 0.1
    <br> 낮은 만큼 최소 오류 값을 찾아 예측성능이 높아질 수 있음. 시간이 많이 소요
- subsample	: 개별 트리가 학습에 사용하는 데이터 샘플링 비율(0~1). default=1 (전체 데이터 학습)
    <br> 이 값을 조절하여 트리 간의 상관도를 줄일 수 있음

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model_gb = GradientBoostingClassifier()
model_gb.fit(X_train, y_train)
model_gb.score(X_train, y_train), model_rf.score(X_test, y_test)

In [ ]:
model_gb = GradientBoostingClassifier()
parameters = {
    'n_estimators' : [50, 100, 300],
    'learning_rate' : [0.01, 0.05, 0.1],
    'max_depth':[3, 4, 5, 6]
}
model_gb_grid = GridSearchCV(model_gb, param_grid=parameters, cv=5,  verbose=1)
model_gb_grid.fit(X_train, y_train)
model_gb_grid.best_params_

In [ ]:
model_gb = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, max_depth=5)
model_gb.fit(X_train, y_train)
model_gb.score(X_test, y_test)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, model_gb.predict(X_test)))

In [ ]:
ftr_importances = pd.Series(model_gb.feature_importances_ , index=X_train.columns  )
plt.figure(figsize=(8,6))
plt.title('Feature importances')
sns.barplot(x=ftr_importances , y = ftr_importances.index)
plt.show()

# XGBoost
- 트리 기반의 알고리즘의 앙상블 학습에서 각광받는 알고리즘 중 하나
- GBM에 기반하고 있지만, GBM의 단점인 느린 수행시간, 과적합 규제 등을 해결한 알고리즘
- 과적합 규제(Overfitting Regularization)


In [ ]:
# pip list

### 설치 

- sklearn 에서 지원되지 않으므로 별도 설치가 필요 
- 관리자 버전으로 설치 
- pip install xgboost
- conda install -c anaconda py-xgboost

### 주요 파라미터 
: 트리 최적화, 부스팅, regularization 등과 관련된 파라미터를 지칭

In [ ]:
import xgboost
from xgboost import plot_importance, XGBClassifier 

xgboost.__version__

### 데이터 셋 - 위스콘신 Breast Cancer 

In [ ]:
from sklearn.datasets import load_breast_cancer

dataset = load_breast_cancer()
X_features= dataset.data
y_label = dataset.target

cancer_df = pd.DataFrame(data=X_features, columns=dataset.feature_names)
cancer_df['target']= y_label

print(cancer_df.shape)
cancer_df.head(3)

In [ ]:
cancer_df['target'].value_counts()

In [ ]:
cancer_df.columns

In [ ]:
# cancer_df.isnull().sum()
cancer_df.isnull().sum().sum()

In [ ]:
cancer_df.describe()

### 데이타셋 분리

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X_features, y_label,
                                         test_size=0.2, random_state=156 )
print(X_train.shape , X_test.shape)

### 모델 생성 및 학습

In [ ]:
# model_xgb = XGBClassifier()
# 오류 발생시 아래와 같이 변경 
model_xgb = XGBClassifier(objective='reg:squarederror', n_estimators=100)
model_xgb.fit(X_train,y_train)
model_xgb.score(X_test, y_test)

In [ ]:
print(classification_report(y_test, model_xgb.predict(X_test)))
confusion_matrix(y_test, model_xgb.predict(X_test))

In [ ]:
model_xgb = XGBClassifier(n_estimators=10)
params = {
    'max_depth':[5, 7, 9],
    'learning_rate':[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
}
model_xgb_gridcv = GridSearchCV(model_xgb, param_grid=params)
model_xgb_gridcv.fit(X_train,
           y_train,
           eval_set=[(X_test, y_test)],
           eval_metric="error")
print('GridSearchCV 최적 파라미터:', model_xgb_gridcv.best_params_)

In [ ]:
model_xgb = XGBClassifier(objective='reg:squarederror', n_estimators=10, learning_rate=0.3, max_depth=5)
model_xgb.fit(X_train,y_train)
model_xgb.score(X_test, y_test)

In [ ]:
ftr_importance_values = model_xgb.feature_importances_
ftr_importances = pd.Series(ftr_importance_values, index=dataset.feature_names)
ftr_importances = ftr_importances.sort_values(ascending=False)
ftr_importances

In [ ]:
sns.barplot(x=ftr_importances, y=ftr_importances.index)